In [ ]:
!pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [1]:
# to monitor the progress
from tqdm import tqdm
import time
# basic dependency
import numpy as np
import pandas as pd
import random
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torch import nn
from torch.utils.data import random_split
from torch import optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [21]:

class MLP(nn.Module):
        # 3 features, thus input size=3
        # output size = 3
        
    def __init__(self, num_of_features=3):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc35 = nn.Linear(32, 24)
        self.fc4 = nn.Linear(24, 16)
        self.fc5 = nn.Linear(16, 8)
        self.fc6 = nn.Linear(8, 3)

    def forward(self, x):
        # Hidden layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc35(x)
        #x = F.relu(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)             
        return x
     

In [12]:
'''
class MLP(nn.Module):
        # 3 features, thus input size=3
        # output size = 3
    def __init__(self, num_of_features=3):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 27)
        self.fc4 = nn.Linear(27, 3)

    def forward(self, x):
        # Hidden layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x
'''

In [4]:
class MLP_score(nn.Module):
        # 6 features, thus input size=3
        # output size = 1
    def __init__(self, num_of_features=6):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 27)
        self.fc4 = nn.Linear(27, 1)

    def forward(self, x):
        # Hidden layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [3]:
def train(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in enumerate(dataloader):
        opt.zero_grad()
        x, y = data
        predict = model(x.float())
        labels = y.float()
        #print(predict,'/',labels)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count

In [6]:
def train_score(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in enumerate(dataloader):
        opt.zero_grad()
        x, y = data
        predict = model(x.float())
        labels = y.float()
        labels = labels.unsqueeze(1)
        #print(predict,'/',labels)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count

In [13]:
# Normalization function
vcpu_scaler = lambda x : (x-1)/(90-1)
ram_scaler = lambda x : (x-1)/(180-1)
volume_scaler = lambda x : (x-1)/(900-1)

# Train size 10000

In [22]:
df_x_test = pd.read_csv('./data/offer_ML/x_test_200_v1.csv',index_col=0).reset_index(drop=True)
df_x_train = pd.read_csv('./data/offer_ML/x_train_104_v1.csv',index_col=0).reset_index(drop=True)
df_y_test = pd.read_csv('./data/offer_ML/v3_y_test_200_v1.csv',index_col=0).reset_index(drop=True)
df_y_train = pd.read_csv('./data/offer_ML/v3_y_train_104_v1.csv',index_col=0).reset_index(drop=True)

In [23]:
df_x_test[['vcpu']] = df_x_test[['vcpu']].apply(vcpu_scaler)
df_x_test[['ram']] = df_x_test[['ram']].apply(ram_scaler)
df_x_test[['volume']] = df_x_test[['volume']].apply(volume_scaler)

df_x_train[['vcpu']] = df_x_train[['vcpu']].apply(vcpu_scaler)
df_x_train[['ram']] = df_x_train[['ram']].apply(ram_scaler)
df_x_train[['volume']] = df_x_train[['volume']].apply(volume_scaler)

df_y_test[['final_vcpu']] = df_y_test[['final_vcpu']].apply(vcpu_scaler)
df_y_test[['final_ram']] = df_y_test[['final_ram']].apply(ram_scaler)
df_y_test[['final_volume']] = df_y_test[['final_volume']].apply(volume_scaler)

df_y_train[['final_vcpu']] = df_y_train[['final_vcpu']].apply(vcpu_scaler)
df_y_train[['final_ram']] = df_y_train[['final_ram']].apply(ram_scaler)
df_y_train[['final_volume']] = df_y_train[['final_volume']].apply(volume_scaler)

In [24]:
list_x_test = df_x_test.values.tolist()
list_x_train = df_x_train.values.tolist()
list_y_test = df_y_test.values.tolist()
list_y_train = df_y_train.values.tolist()

In [25]:
x_train = torch.tensor(list_x_train)
x_test = torch.tensor(list_x_test)
y_train = torch.tensor(list_y_train)
y_test = torch.tensor(list_y_test)
train_ids = TensorDataset(x_train, y_train) 
test_ids = TensorDataset(x_test, y_test) 

In [26]:
# Train 
MLP_fuzzy = MLP(num_of_features=3)
# settings
criterion = nn.MSELoss()
opt = optim.Adam(MLP_fuzzy.parameters(), lr=0.001)
epoch = 200
# load data
train_dataloader = DataLoader(dataset=train_ids, batch_size=64, shuffle = True)
test_dataloader = DataLoader(dataset=test_ids, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in tqdm(range(epoch)):
    loss_train = train(MLP_fuzzy, train_dataloader, criterion, opt)
    print("epoch%d   loss: %.5f "\
      % (i+1, loss_train))
torch.save(MLP_fuzzy.state_dict(), "./models/MLP_fuzzy_v3_v1_4layer.pth")

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

start training: 


  0%|▍                                                                                 | 1/200 [00:00<01:47,  1.85it/s]

epoch1   loss: 0.04075 


  1%|▊                                                                                 | 2/200 [00:01<01:44,  1.89it/s]

...

epoch198   loss: 0.00954 


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [01:43<00:00,  1.93it/s]

epoch199   loss: 0.00954 


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:43<00:00,  1.93it/s]

epoch200   loss: 0.00954 


In [27]:
predict_list = []
for x_test, y_test in test_ids:
    
    MLP_fuzzy.eval()
    
    predict = MLP_fuzzy(x_test.float())
    predict = predict.tolist()
    predict[0] = round((90-1)*predict[0]+1)
    predict[1] = round((180-1)*predict[1]+1)
    predict[2] = round((900-1)*predict[2]+1)
    for i in range(len(predict)):
        if predict[i]<=0:
            predict[i]=1
    
    labels = y_test.float()
    labels = labels.tolist()
    labels[0] = round((90-1)*labels[0]+1)
    labels[1] = round((180-1)*labels[1]+1)
    labels[2] = round((900-1)*labels[2]+1)
    predict_list.append(predict)
    print(predict,' vs ' ,labels)

[35, 52, 441]  vs  [9, 50, 400]
[34, 58, 300]  vs  [7, 35, 230]
[38, 74, 455]  vs  [13, 108, 400]
[38, 75, 392]  vs  [13, 110, 320]
[40, 77, 552]  vs  [14, 112, 520]
[35, 52, 438]  vs  [16, 55, 400]
[40, 71, 643]  vs  [18, 108, 640]
[35, 70, 195]  vs  [8, 51, 80]
[37, 72, 388]  vs  [15, 104, 320]
[35, 54, 414]  vs  [19, 65, 370]
[35, 68, 273]  vs  [9, 49, 180]
[42, 81, 697]  vs  [19, 61, 700]
[42, 84, 691]  vs  [19, 65, 690]
[34, 58, 289]  vs  [10, 36, 220]
[40, 74, 658]  vs  [20, 112, 660]
[42, 86, 643]  vs  [20, 68, 630]
[43, 93, 648]  vs  [20, 77, 630]
[43, 80, 789]  vs  [41, 119, 820]
[44, 87, 835]  vs  [34, 68, 870]
[40, 88, 485]  vs  [33, 73, 430]
[43, 90, 677]  vs  [34, 73, 670]
[39, 96, 331]  vs  [41, 85, 530]
[34, 56, 261]  vs  [12, 35, 180]
[35, 60, 336]  vs  [13, 40, 280]
[36, 61, 457]  vs  [42, 99, 420]
[37, 79, 310]  vs  [42, 63, 520]
[37, 80, 263]  vs  [42, 65, 460]
[39, 88, 356]  vs  [42, 74, 570]
[39, 92, 367]  vs  [42, 79, 580]
[44, 95, 751]  vs  [36, 79, 760]
[45, 97,

In [28]:
df_predict_test_200_train_104 = pd.DataFrame(predict_list)
df_predict_test_200_train_104.columns = ['predict_vcpu','predict_ram','predict_volume']
df_predict_test_200_train_104.to_csv('./data/offer_ML/v3_predict_test_200_train_104_v1_7layer.csv')

In [ ]:
df_predict_test_200_train_104 = pd.DataFrame(predict_list)
df_predict_test_200_train_104.columns = ['predict_vcpu','predict_ram','predict_volume']
df_predict_test_200_train_104.to_csv('./data/offer_ML/v3_predict_test_200_train_104_v1_deepNN.csv')

In [20]:
df_predict_test_200_train_104 = pd.DataFrame(predict_list)
df_predict_test_200_train_104.columns = ['predict_vcpu','predict_ram','predict_volume']
df_predict_test_200_train_104.to_csv('./data/offer_ML/v3_predict_test_200_train_104_v1_4layer.csv')

# Train size 20000

In [10]:
df_x_test = pd.read_csv('./data/offer_ML/x_test_200.csv',index_col=0).reset_index(drop=True)
df_x_train = pd.read_csv('./data/offer_ML/x_train_204.csv',index_col=0).reset_index(drop=True)
df_y_test = pd.read_csv('./data/offer_ML/v4_y_test_200.csv',index_col=0).reset_index(drop=True)
df_y_train = pd.read_csv('./data/offer_ML/v4_y_train_204.csv',index_col=0).reset_index(drop=True)

In [11]:
df_x_test[['vcpu']] = df_x_test[['vcpu']].apply(vcpu_scaler)
df_x_test[['ram']] = df_x_test[['ram']].apply(ram_scaler)
df_x_test[['volume']] = df_x_test[['volume']].apply(volume_scaler)

df_x_train[['vcpu']] = df_x_train[['vcpu']].apply(vcpu_scaler)
df_x_train[['ram']] = df_x_train[['ram']].apply(ram_scaler)
df_x_train[['volume']] = df_x_train[['volume']].apply(volume_scaler)

df_y_test[['final_vcpu']] = df_y_test[['final_vcpu']].apply(vcpu_scaler)
df_y_test[['final_ram']] = df_y_test[['final_ram']].apply(ram_scaler)
df_y_test[['final_volume']] = df_y_test[['final_volume']].apply(volume_scaler)

df_y_train[['final_vcpu']] = df_y_train[['final_vcpu']].apply(vcpu_scaler)
df_y_train[['final_ram']] = df_y_train[['final_ram']].apply(ram_scaler)
df_y_train[['final_volume']] = df_y_train[['final_volume']].apply(volume_scaler)

In [14]:
list_x_test = df_x_test.values.tolist()
list_x_train = df_x_train.values.tolist()
list_y_test = df_y_test.values.tolist()
list_y_train = df_y_train.values.tolist()
x_train = torch.tensor(list_x_train)
x_test = torch.tensor(list_x_test)
y_train = torch.tensor(list_y_train)
y_test = torch.tensor(list_y_test)
train_ids = TensorDataset(x_train, y_train) 
test_ids = TensorDataset(x_test, y_test) 
# Train 
MLP_fuzzy = MLP(num_of_features=3)
# settings
criterion = nn.MSELoss()
opt = optim.Adam(MLP_fuzzy.parameters(), lr=0.001)
epoch = 200
# load data
train_dataloader = DataLoader(dataset=train_ids, batch_size=64, shuffle = True)
test_dataloader = DataLoader(dataset=test_ids, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in tqdm(range(epoch)):
    loss_train = train(MLP_fuzzy, train_dataloader, criterion, opt)
    print("epoch%d   loss: %.5f "\
      % (i+1, loss_train))
torch.save(MLP_fuzzy.state_dict(), "./models/MLP_fuzzy.pth")

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

start training: 


  0%|▍                                                                                 | 1/200 [00:00<02:18,  1.43it/s]

epoch1   loss: 0.01660 


  1%|▊                                                                                 | 2/200 [00:01<02:13,  1.48it/s]

...

epoch198   loss: 0.01362 


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [02:05<00:00,  1.65it/s]

epoch199   loss: 0.01362 


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:05<00:00,  1.59it/s]

epoch200   loss: 0.01362 


In [30]:
predict_list = []
time_start=time.time()
for x_test, y_test in test_ids:
    
    MLP_fuzzy.eval()
    
    predict = MLP_fuzzy(x_test.float())
    predict = predict.tolist()
    predict[0] = round(90*predict[0])
    predict[1] = round(180*predict[1])
    predict[2] = round(900*predict[2])
    
    labels = y_test.float()
    labels = labels.tolist()
    labels[0] = round(90*labels[0])
    labels[1] = round(180*labels[1])
    labels[2] = round(900*labels[2])
    
    predict_list.append(predict)
    #print(predict,' vs ' ,labels)
time_end=time.time()
print('time cost',time_end-time_start,'s')

time cost 0.07139277458190918 s


In [17]:
df_predict_test_200_train_204 = pd.DataFrame(predict_list)
df_predict_test_200_train_204.columns = ['predict_vcpu','predict_ram','predict_volume']
df_predict_test_200_train_204.to_csv('./data/offer_ML/v4_predict_test_200_train_204.csv')

# Train size 40000

In [ ]:
df_x_test = pd.read_csv('./data/offer_ML/x_test_200.csv',index_col=0).reset_index(drop=True)
df_x_train = pd.read_csv('./data/offer_ML/x_train_404.csv',index_col=0).reset_index(drop=True)
df_y_test = pd.read_csv('./data/offer_ML/v4_y_test_200.csv',index_col=0).reset_index(drop=True)
df_y_train = pd.read_csv('./data/offer_ML/v4_y_train_404.csv',index_col=0).reset_index(drop=True)

In [ ]:
df_x_test[['vcpu']] = df_x_test[['vcpu']].apply(vcpu_scaler)
df_x_test[['ram']] = df_x_test[['ram']].apply(ram_scaler)
df_x_test[['volume']] = df_x_test[['volume']].apply(volume_scaler)

df_x_train[['vcpu']] = df_x_train[['vcpu']].apply(vcpu_scaler)
df_x_train[['ram']] = df_x_train[['ram']].apply(ram_scaler)
df_x_train[['volume']] = df_x_train[['volume']].apply(volume_scaler)

df_y_test[['final_vcpu']] = df_y_test[['final_vcpu']].apply(vcpu_scaler)
df_y_test[['final_ram']] = df_y_test[['final_ram']].apply(ram_scaler)
df_y_test[['final_volume']] = df_y_test[['final_volume']].apply(volume_scaler)

df_y_train[['final_vcpu']] = df_y_train[['final_vcpu']].apply(vcpu_scaler)
df_y_train[['final_ram']] = df_y_train[['final_ram']].apply(ram_scaler)
df_y_train[['final_volume']] = df_y_train[['final_volume']].apply(volume_scaler)

In [12]:
list_x_test = df_x_test.values.tolist()
list_x_train = df_x_train.values.tolist()
list_y_test = df_y_test.values.tolist()
list_y_train = df_y_train.values.tolist()
x_train = torch.tensor(list_x_train)
x_test = torch.tensor(list_x_test)
y_train = torch.tensor(list_y_train)
y_test = torch.tensor(list_y_test)
train_ids = TensorDataset(x_train, y_train) 
test_ids = TensorDataset(x_test, y_test) 
# Train 
MLP_fuzzy = MLP(num_of_features=3)
# settings
criterion = nn.MSELoss()
opt = optim.Adam(MLP_fuzzy.parameters(), lr=0.001)
epoch = 200
# load data
train_dataloader = DataLoader(dataset=train_ids, batch_size=64, shuffle = True)
test_dataloader = DataLoader(dataset=test_ids, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in tqdm(range(epoch)):
    loss_train = train(MLP_fuzzy, train_dataloader, criterion, opt)
    print("epoch%d   loss: %.5f "\
      % (i+1, loss_train))
torch.save(MLP_fuzzy.state_dict(), "./models/MLP_fuzzy.pth")

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

start training: 


  0%|▍                                                                                 | 1/200 [00:00<03:01,  1.09it/s]

epoch1   loss: 1948.72754 


  1%|▊                                                                                 | 2/200 [00:01<02:58,  1.11it/s]

...

epoch198   loss: 311.44482 


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [02:56<00:00,  1.14it/s]

epoch199   loss: 312.16846 


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:57<00:00,  1.13it/s]

epoch200   loss: 311.79791 


In [8]:
predict_list = []
for x_test, y_test in test_ids:
    MLP_fuzzy.eval()
    predict = MLP_fuzzy(x_test.float())
    labels = y_test.float()
    predict_list.append(predict.tolist())
    print(predict.tolist(),' vs ' ,labels.tolist())

[23.42633628845215, 68.52094268798828, 871.1996459960938]  vs  [16.0, 52.0, 870.0]
[62.441558837890625, 98.82212829589844, 780.480712890625]  vs  [58.0, 76.0, 780.0]
[30.476951599121094, 76.22879028320312, 771.3834838867188]  vs  [47.0, 100.0, 770.0]
[46.52101135253906, 92.6486587524414, 880.0518798828125]  vs  [36.0, 63.0, 880.0]
[21.12329864501953, 80.03665924072266, 611.8522338867188]  vs  [11.0, 113.0, 610.0]
[46.66582107543945, 82.6421127319336, 701.6405639648438]  vs  [39.0, 112.0, 700.0]
[29.207223892211914, 77.4857406616211, 701.6616821289062]  vs  [46.0, 106.0, 700.0]
[46.07639694213867, 89.2911605834961, 850.4126586914062]  vs  [36.0, 118.0, 850.0]
[24.61075782775879, 85.2770004272461, 810.5110473632812]  vs  [18.0, 117.0, 810.0]
[49.77777099609375, 76.49885559082031, 582.7183837890625]  vs  [45.0, 105.0, 580.0]
[39.36717987060547, 79.24237060546875, 831.0255737304688]  vs  [58.0, 101.0, 830.0]
[23.28476905822754, 76.21315002441406, 890.6157836914062]  vs  [13.0, 80.0, 890.0]

In [14]:
df_predict_test_train_404 = pd.DataFrame(predict_list)
df_predict_test_train_404.to_csv('./data/offer_ML/v4_predict_test_200_train_404.csv')

# Score train

In [40]:
df_test = pd.read_csv('./data/score_test_20.csv',index_col=0).reset_index(drop=True)
df_train = pd.read_csv('./data/score_train_104.csv',index_col=0).reset_index(drop=True)

df_x_test = df_test.iloc[:,:6]
df_x_train = df_train.iloc[:,:6]
df_y_test = df_test.iloc[:,6]
df_y_train = df_train.iloc[:,6]

In [41]:
list_x_test = df_x_test.values.tolist()
list_x_train = df_x_train.values.tolist()
list_y_test = df_y_test.values.tolist()
list_y_train = df_y_train.values.tolist()
x_train = torch.tensor(list_x_train)
x_test = torch.tensor(list_x_test)
y_train = torch.tensor(list_y_train)
y_test = torch.tensor(list_y_test)
train_ids = TensorDataset(x_train, y_train) 
test_ids = TensorDataset(x_test, y_test) 
# Train 
MLP_scores = MLP_score(num_of_features=6)
# settings
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
opt = optim.Adam(MLP_scores.parameters(), lr=0.0001)
epoch = 200
# load data
train_dataloader = DataLoader(dataset=train_ids, batch_size=64, shuffle = True)
test_dataloader = DataLoader(dataset=test_ids, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in tqdm(range(epoch)):
    loss_train = train_score(MLP_scores, train_dataloader, criterion, opt)
    print("epoch%d   loss: %.5f "\
      % (i+1, loss_train))
torch.save(MLP_scores.state_dict(), "./models/MLP_score.pth")

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

start training: 


  0%|▍                                                                                 | 1/200 [00:03<11:26,  3.45s/it]

epoch1   loss: 10.45976 


  1%|▊                                                                                 | 2/200 [00:06<10:49,  3.28s/it]

...

epoch198   loss: 1.93396 


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [09:59<00:02,  2.98s/it]

epoch199   loss: 1.95186 


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [10:02<00:00,  3.01s/it]

epoch200   loss: 1.92983 


In [42]:
predict_list = []
for x_test, y_test in test_ids:
    MLP_scores.eval()
    predict = MLP_scores(x_test.float())
    labels = y_test.float()
    predict_list.append(predict.tolist())
    print(predict.tolist(),' vs ' ,labels.tolist())

[47.37700271606445]  vs  45.854373931884766
[47.39314270019531]  vs  45.9044303894043
[47.40935134887695]  vs  45.95463562011719
[47.42547607421875]  vs  46.005001068115234
[47.441680908203125]  vs  46.05662536621094
[47.457847595214844]  vs  46.11140060424805
[47.4739990234375]  vs  46.168601989746094
[47.49015808105469]  vs  46.22602462768555
[47.506324768066406]  vs  46.28368377685547
[47.52251052856445]  vs  46.34158706665039
[47.53868103027344]  vs  46.39974594116211
[47.55485534667969]  vs  46.45936584472656
[47.57103729248047]  vs  46.52040481567383
[47.58719253540039]  vs  46.58628463745117
[47.603328704833984]  vs  46.65251159667969
[47.61952590942383]  vs  46.719093322753906
[47.63568878173828]  vs  46.786048889160156
[47.6518440246582]  vs  46.8533935546875
[47.667999267578125]  vs  46.92285919189453
[47.68415832519531]  vs  46.99349594116211
[47.700355529785156]  vs  47.06819152832031
[47.716529846191406]  vs  47.1446647644043
[47.73271942138672]  vs  47.22166061401367
[47.